In [1]:
from binance_wrapper import BinanceWrapper
from analytics import Analytics

import matplotlib.pyplot as plt

/Users/miller/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/miller/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Requesting data

In [2]:
# Products being requested
symbols = ['BTCUSDT', 'ETHUSDT', ]

# Time period being requested
start_time, end_time = '2025-08-01', '2025-08-09'

# Granularity of the requested data
interval = '1h'

In [3]:
data = BinanceWrapper().get_data(symbols=symbols,
                                 start_time=start_time,
                                 end_time=end_time,
                                 interval=interval)

### Computing analytics

In [4]:
bitcoin_df = data['BTCUSDT']

window = 10

bitcoin_df = Analytics.moving_average(df=bitcoin_df, column='CLOSE', window=window)

bitcoin_df = Analytics.exponential_ma(df=bitcoin_df, column='CLOSE', window=window)

bitcoin_df = Analytics.bollinger_bands(df=bitcoin_df, column='CLOSE', window=window)

In [5]:
cols = ['CLOSE', f'MA{window}_CLOSE', f'BB_Upper_{window}', f'BB_Lower_{window}']

bitcoin_df.dropna(subset=cols, inplace=True)

### Computing interactive plots

In [8]:
from IPython.display import display, clear_output
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import ipywidgets as widgets

plt.style.use("dark_background")

line_width = 2

window_slider = widgets.IntSlider(
    min=1, 
    max=30, 
    step=1, 
    value=5, 
    description="Moving Average Size", 
    continuous_update=False,
    style={'description_width': '180px'},  # widen description area
    layout=widgets.Layout(width='400px')   # widen whole widget
)

bb_factor_slider = widgets.FloatSlider(
    min=1.0, 
    max=5.0, 
    step=0.1,
    value=2.0, 
    description="Bollinger Bands Factor", 
    continuous_update=False,
    style={'description_width': '180px'},
    layout=widgets.Layout(width='400px')
)

output = widgets.Output()

def update_plot(change):
    with output:
        clear_output(wait=True)
        df_ = bitcoin_df.copy()
        w = window_slider.value
        factor = bb_factor_slider.value
        
        # Compute indicators
        df_ = Analytics.moving_average(df_, 'CLOSE', w)
        df_ = Analytics.bollinger_bands(df_, 'CLOSE', w, factor)
        
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Plot lines
        ax.plot(df_['CLOSE'], label='Close Price', color='#00ffec', linewidth=line_width)
        ax.plot(df_[f'MA{w}_CLOSE'], label=f'MA{w}', color='lavender', linewidth=line_width)
        ax.plot(df_[f'BB_Upper_{w}'], label='Bollinger Upper', linestyle='--', color='red', linewidth=line_width)
        ax.plot(df_[f'BB_Lower_{w}'], label='Bollinger Lower', linestyle='--', color='red', linewidth=line_width)
        
        # Fill area between Bollinger Bands
        ax.fill_between(
            df_.index,
            df_[f'BB_Upper_{w}'],
            df_[f'BB_Lower_{w}'],
            color='red',
            alpha=0.2
        )
        
        # Bloomberg-like grid
        ax.grid(color='#444444', linestyle='--', linewidth=0.5)
        
        # Format y-axis to thousands
        ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x/1e3:.1f}'))
        
        # Remove x-axis ticks and labels
        ax.set_xticks([])
        ax.set_xticklabels([])
        
        # Labels and legend
        ax.set_ylabel('Price (k$)', fontsize=12, fontweight='bold', color='white')
        ax.legend(facecolor='#111111', edgecolor='white', fontsize=10)
        ax.set_title('Bitcoin Close Price & Trading Indicators', fontsize=14, fontweight='bold', color='white')
        
        plt.tight_layout()
        plt.show()

window_slider.observe(update_plot, names='value')
bb_factor_slider.observe(update_plot, names='value')

# Place sliders side-by-side using HBox
slider_box = widgets.HBox([window_slider, bb_factor_slider])

display(slider_box, output)

update_plot(None)  # initial plot


Output()